In [2]:
import glob, h5py, pickle, cv2, re
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import mat73
from IPython.display import clear_output
from scipy.io import loadmat


width = 224
height = 224
channels = 3

num_class = 10
sub = 'imit_ASL_' + str(num_class) + '_skel'
filename = '/mnt/HDD04/ICASSP_2023/Scripts/datasets/'+sub+'.pkl'
print('File to be created: ' + filename)

File to be created: /mnt/HDD04/ICASSP_2023/Scripts/datasets/imit_ASL_10_skel.pkl


In [3]:
env_path = '/mnt/HDD04/ASL_10_imit/outputs/envelopes_sim/'
env_threshold = 2e3
err_path = '/mnt/HDD04/ASL_10_imit/scripts/err_files.txt'
vid_path = '/mnt/HDD04/ASL_10_imit/outputs/webcam/cut/'
skel_path = env_path.replace('envelopes_sim', 'skeletons_mat')
md_path = vid_path.replace('webcam', 'microDoppler_real')

md_files = glob.glob(md_path + '*png')
env_files = glob.glob(env_path + '*txt')
fnames = [m.split('/')[-1][:-4] for m in md_files]
print('Num. env files:', len(env_files))
print('Num. files:', len(fnames))

Num. env files: 0
Num. files: 1980


In [4]:
# import itertools
# clas_dict = dict()
# for e in env_files:
#     fname = e.split('/')[-1]
#     underscore_idx = [m.start() for m in re.finditer('_', fname)]
#     word_id = fname.find('word')
#     clas = fname[word_id+4:underscore_idx[-1]]
#     if clas not in clas_dict:
#         clas_dict[clas] = 1
#     else:
#         clas_dict[clas] += 1
# sorted_clas_dict = {k: v for k, v in sorted(clas_dict.items(), key=lambda item: item[1])}
# reverse =dict(reversed(list(sorted_clas_dict.items())))
# # select the ones with most samples
# sorted_clas_dict = dict(itertools.islice(reverse.items(), num_class))
# print(sorted_clas_dict)
# select_classes = sorted_clas_dict.keys()
# print('select_classes:', select_classes)

In [5]:
new_classes = {'125': 0, '121': 1, '122': 2, '123': 3, '117': 4,
               '119': 5, '131': 6, '128': 7, '129': 8, '118': 9}

In [6]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [7]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [8]:
def FrameCapture_gray(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.cvtColor(cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR), cv2.COLOR_BGR2GRAY)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
#     result = np.expand_dims(np.array([frames[i] for i in range(len(frames))]), -1)
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [9]:
def pad_video(vid, num_pad):
#     print(vid.shape)
#     print(vid[0].shape)
#     print(vid[1].shape)
    vid = np.concatenate([vid, np.tile(vid[-1,:,:], [num_pad, 1, 1, 1])])
#     vid = np.concatenate([vid, np.tile(vid[-1,:,:], [num_pad, 1, 1])])
    return vid

In [10]:
with open(err_path, 'r') as err:
    err_files = []
    for line in err:
        err_files.append(line.split(",")[0][:-1])
err_files[:5]

['2022_10_13_12_52_55_subjLadi_Exp0_class125_part5_word125_cnt8',
 '2022_10_13_13_00_54_subjLadi_Exp0_class122_part1_word122_cnt9',
 '2022_10_13_13_04_32_subjLadi_Exp0_class123_part3_word123_cnt8',
 '2022_10_13_13_04_32_subjLadi_Exp0_class123_part4_word123_cnt9',
 '2022_10_13_13_13_51_subjLadi_Exp0_class119_part4_word119_cnt12']

In [11]:
# remove md files which doesn't have corresponding vid file due to labeling error
for f in fnames:
    if f in err_files:
        fnames.remove(f)
print(len(fnames))

for f in fnames[:5]:
    print(f)

1924
2022_10_13_16_12_38_subjEmre_Exp0_class128_part2_word128_cnt5
2022_10_13_13_26_55_subjLadi_Exp0_class118_part4_word118_cnt10
2022_10_13_15_32_17_subjSean_Exp0_class125_part1_word125_cnt11
2022_10_13_16_19_32_subjEmre_Exp0_class118_part1_word118_cnt6
2022_10_13_16_16_32_subjEmre_Exp0_class129_part5_word129_cnt3


In [12]:
train_files, test_files = train_test_split(fnames, test_size=0.2, random_state=1)

In [13]:
train_images = []
train_vids = []
train_skels = []
train_labels = []
max_vid_len = 150 # 5 sec x 30 frames
train_mask = np.zeros((len(train_files), max_vid_len))

for i, f in enumerate(train_files):
    
    underscore_idx = [m.start() for m in re.finditer('_', f)]
    word_id = f.find('word')
    clas = f[word_id+4:underscore_idx[-1]]
    
    if clas not in new_classes:
        continue
        
    # read envelope
#     env = load_file(e)
#     if sum(env[0]) < env_threshold:
#         continue
        
    # read video
#     vid_name = vid_path + f + '.avi'
# #     vid = FrameCapture_gray(vid_name)
#     vid = FrameCapture(vid_name)
#     num_frames = vid.shape[0]
#     if num_frames > max_vid_len:
#         vid = vid[:max_vid_len]
#     elif num_frames < max_vid_len:
#         num_pad = max_vid_len - num_frames
#         train_mask[i, -num_pad:] = 1
#         vid = pad_video(np.array(vid), num_pad)
#     train_vids.append(vid[::3])
    
    # read skeleton
    skel_name = skel_path + f + '.mat'
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < max_vid_len:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [max_vid_len - skel.shape[0], 1, 1])])
    elif skel.shape[0] > max_vid_len:
        skel = skel[:max_vid_len]
    train_skels.append(skel)
    
    # read microDoppler
#     md_name = md_path + f + '.png'
#     img = cv2.imread(md_name)
#     img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing training file: '+ str(i+1) + '/' + str(len(train_files)))
#     train_images.append(img)
    train_labels.append(new_classes[clas])
#     if i == 2:
#         break

train_labels = to_categorical(np.array(train_labels), num_class)
# train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
# train_images = np.swapaxes(train_images, 1, 3)
# train_vids = np.array(train_vids)/255.
train_skels = np.array(train_skels)
# print(train_images.shape)
print(train_skels.shape)
print(train_labels.shape)
# print(train_vids.shape)
print('max_vid_len:', max_vid_len)

Processing training file: 1539/1539
(1539, 150, 3, 42)
(1539, 10)
max_vid_len: 150


In [14]:
# [i.shape for i in train_skels if len(i) != 150]

In [15]:
test_images = []
test_vids = []
test_skels = []
test_labels = []
test_mask = np.zeros((len(test_files), max_vid_len))

for i, f in enumerate(test_files):
    
    underscore_idx = [m.start() for m in re.finditer('_', f)]
    word_id = f.find('word')
    clas = f[word_id+4:underscore_idx[-1]]
    
    if clas not in new_classes:
        continue
    
    # read envelope
#     env = load_file(e)
#     if sum(env[0]) < env_threshold:
#         continue
        
    # read video
#     vid_name = vid_path + f + '.avi'
#     vid = FrameCapture(vid_name)
# #     vid = FrameCapture_gray(vid_name)
#     num_frames = vid.shape[0]
#     if num_frames > max_vid_len:
#         max_vid_len = num_frames
#         vid = vid[:max_vid_len]
#     elif num_frames < max_vid_len:
#         num_pad = max_vid_len - num_frames
#         test_mask[i, -num_pad:] = 1
#         vid = pad_video(np.array(vid), num_pad)
#     test_vids.append(vid[::3])
    
    # read skeleton
    skel_name = skel_path + f + '.mat'
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < max_vid_len:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [max_vid_len - skel.shape[0], 1, 1])])
    elif skel.shape[0] > max_vid_len:
        skel = skel[:max_vid_len]
    test_skels.append(skel)
    
    # read microDoppler
#     md_name = md_path + f + '.png'
#     img = cv2.imread(md_name)
#     img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing testing file: '+ str(i+1) + '/' + str(len(test_files)))
#     test_images.append(img)
    test_labels.append(new_classes[clas])

test_labels = to_categorical(np.array(test_labels), num_class)
# test_images = np.swapaxes(np.array(test_images), 1, 2).reshape(len(test_images), width, height, channels)/255.
# test_images = np.swapaxes(test_images, 1, 3)
test_vids = np.array(test_vids)/255.
test_skels = np.array(test_skels)
# print(test_images.shape)
print(test_skels.shape)
print(test_labels.shape)
# print(test_vids.shape)

Processing testing file: 385/385
(385, 150, 3, 42)
(385, 10)


In [16]:
[i.shape for i in test_vids if len(i) != 50]

[]

In [17]:
data = [train_files, train_skels, train_labels, train_mask,
        test_files, test_skels, test_labels, test_mask]
# data = [train_images, train_labels, train_files,
#         test_images, test_labels, test_files]

with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

/mnt/HDD04/ICASSP_2023/Scripts/datasets/imit_ASL_10_skel.pkl created.
